In [1]:
%load_ext autoreload
%autoreload 2
from keras.applications.vgg16 import VGG16

import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
from keras.preprocessing import image
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

import models.simpleConv as simpleConv
import models.crazyNet as crazyNet
import models.newModel as newModel
from keras.applications.resnet50 import ResNet50
# import models.betterConv as betterConv
from imgaug import augmenters as iaa

import h5py
from imgaug import augmenters as iaa
from keras.utils import np_utils

import pickle

import utils
import preprocessing as pp
pp.init()

allTrainingFolders = [x for x in os.listdir(pp.trainPath) if x.startswith('n')]
np.random.shuffle(allTrainingFolders)
num = 50
trainingFolders = allTrainingFolders[0:num]

# with open('trainingFolderOrder.text', 'w') as f:
#     f.write(" ".join(trainingFolders))

  


Using TensorFlow backend.


Checking to make sure output directories are created..
..done


In [24]:
pp.trainPath = 'E:\\SCHOOL CMPS 240\\ILSVRC\Data\\CLS-LOC\\train'

allTrainingFolders = [x for x in os.listdir(pp.trainPath) if x.startswith('n')]
np.random.shuffle(allTrainingFolders)
num = 10
trainingFolders = allTrainingFolders[0:num]

trainingFolders

['n03179701',
 'n01980166',
 'n02268443',
 'n02108915',
 'n02865351',
 'n09428293',
 'n02895154',
 'n02825657',
 'n01608432',
 'n02782093']

In [44]:
arr =[pp.translateID(x) for x in trainingFolders]

for x, y in zip(arr, trainingFolders):
    print(x, y)

turnstile n04501370
potter's wheel n03992509
cassette player n02979186
flat-coated retriever n02099267
axolotl, mud puppy, Ambystoma mexicanum n01632777
triceratops n01704323
cabbage butterfly n02280649
drake n01847000
bullet train, bullet n02917067
Kerry blue terrier n02093859
wok n04596742
schipperke n02104365
bobsled, bobsleigh, bob n02860847
apiary, bee house n02727426
marmoset n02490219
amphibian, amphibious vehicle n02704792
beaver n02363005
hay n07802026
giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca n02510455
racket, racquet n04039381
sidewinder, horned rattlesnake, Crotalus cerastes n01756291
projectile, missile n04008634
gong, tam-tam n03447721
sports car, sport car n04285008
jeep, landrover n03594945
pitcher, ewer n03950228
china cabinet, china closet n03018349
soup bowl n04263257
German short-haired pointer n02100236
American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier n02093428
lesser panda, red panda, panda,

In [ ]:
parse = False
train = True
saveModel = False
includeAugmented = False
onlyAugmented = False

augments = [iaa.GaussianBlur(3.0)]
h5filename = '{0}-RANDcrazy.h5'.format(num)
h5file = os.path.join(pp.h5Path, h5filename)

if parse:
    if os.path.isfile(os.path.join(pp.h5Path, h5filename)):
        os.remove(os.path.join(pp.h5Path, h5filename))
    pp.parseImages(trainingFolders, h5filename)
#     pp.shuffleH5(h5file)
    #write to text file
    with open(os.path.join('output', 'trainingFolderOrder.text'), 'wb') as fp:     
        pickle.dump(trainingFolders, fp)
    
#read back    
with open (os.path.join('output', 'trainingFolderOrder.text'), 'rb') as fp:     
    trainingFolders = pickle.load(fp)
    
encoder = LabelBinarizer()
##MAKE SURE classLabels is set to ALL the folders you will train on, even if doing in batches
encoder = encoder.fit(trainingFolders)
    
#NOTE: KEEP BATCH SIZE = # of all folders for now because we need to shuffle H5. 


filepath=os.path.join(pp.outputModelPath, "best-epoch-CrazyEditedw50-{epoch:02d}-{val_acc:.2f}.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks = [checkpoint]
epochs = 100
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# adam = keras.optimizers.Adam(lr=0.01, decay=1e-6)
model = crazyNet.build_model3(len(encoder.classes_))
# model = ResNet50(weights=None, include_top ='True', classes=50) 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model = load_model((os.path.join(pp.outputModelPath, 'CrazyNet5Epoch50Class.h5')))


batch_size = 64


##FIT MODEL
if train:
    train_generator = pp.DataGenerator3(h5file, trainingFolders, batch_size=batch_size)
    val_generator = pp.DataGenerator3(h5file, trainingFolders, batch_size=batch_size, isValidation = True)
    print(len(train_generator))
    results = model.fit_generator(train_generator, validation_data = val_generator, epochs=epochs, callbacks = callbacks, verbose=1,
                              use_multiprocessing=True, workers=4)

    if saveModel:
        count = 1
        outputName = ""
        while True:
            k = ''
            if includeAugmented:
                k = 'with-aug'
            if onlyAugmented:
                k = 'aug-only'
            s = pp.baseModelName.format(epoch, count, k) + ".h5" 
            if not s in os.listdir(pp.outputModelPath):         
                outputName = s         
                break     
            else:         
                count+=1 
        print(outputName)
        model.save(os.path.join(pp.outputModelPath, outputName))
        


313
Epoch 1/100


In [10]:
h5db = h5py.File(h5file, 'r') 
h5db.close()

In [4]:
pp.shuffleH5(h5file)

'output\\image-h5\\50-RANDcrazy.h5'

In [11]:
[pp.translateID(X) for X in trainingFolders]

['vine snake',
 'triceratops',
 'quail',
 'tiger shark, Galeocerdo cuvieri',
 'common newt, Triturus vulgaris',
 'house finch, linnet, Carpodacus mexicanus',
 'diamondback, diamondback rattlesnake, Crotalus adamanteus',
 'prairie chicken, prairie grouse, prairie fowl',
 'hornbill',
 'alligator lizard',
 'trilobite',
 'frilled lizard, Chlamydosaurus kingi',
 'Indian cobra, Naja naja',
 'chickadee',
 'magpie',
 'electric ray, crampfish, numbfish, torpedo',
 'goldfish, Carassius auratus',
 'horned viper, cerastes, sand viper, horned asp, Cerastes cornutus',
 'hen',
 'sulphur-crested cockatoo, Kakatoe galerita, Cacatua galerita',
 'bulbul',
 'terrapin',
 'black grouse',
 'black swan, Cygnus atratus',
 'hummingbird',
 'water ouzel, dipper',
 'ringneck snake, ring-necked snake, ring snake',
 'water snake',
 'African chameleon, Chamaeleo chamaeleon',
 'jay',
 'black widow, Latrodectus mactans',
 'night snake, Hypsiglena torquata',
 'axolotl, mud puppy, Ambystoma mexicanum',
 'box turtle, box 

In [13]:
model.save(os.path.join(pp.outputModelPath, 'CrazyNet67Epoch50ClassNewGen.h5'))

In [7]:
# model = load_model((os.path.join(pp.outputModelPath, 'CrazyNet5Epoch50Class.h5')))

h5db = h5py.File(h5file, 'r')
pp.top5accuracies(model, h5db['x_val'][:], h5db['y_val'][:], encoder)
                              

['n01739381' 'n01739381' 'n01739381' ... 'n01784675' 'n01784675'
 'n01784675']


[0.29471383147853736,
 0.4211049284578696,
 0.5003974562798092,
 0.5562400635930048,
 0.6059220985691574]

In [12]:
y_val = [k.decode('utf-8') for k in y_val]
y_val = encoder.fit_transform(y_val)
print(encoder.inverse_transform(y_val))

AttributeError: 'str' object has no attribute 'decode'

In [10]:
y_val

array([b'n01440764', b'n01440764', b'n01440764', ..., b'n01518878',
       b'n01518878', b'n01518878'], dtype='|S9')

In [14]:
h5file

'output\\image-h5\\10-crazy.h5'